## Installation

In [ ]:
pip install wandb numpy pandas matplotlib

## Q1: fashion-MNIST dataset

In [ ]:
import wandb
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

classes = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

def logClassImages(project_name:str):
  wandb.init(project=project_name)
  wandb_image_indices = []

  for classNumber in range(10):
    for j in range(len(y_test)):
      if y_test[j] == classNumber:
        wandb_image_indices.append(x_test[j])
        break

  wandb_images = [wandb.Image(wandb_image_indices[i], caption = classes[i]) for i in range(10)]
  wandb.log({"Sample images for each class": wandb_images})
  wandb.finish()

# logClassImages("da6401_assignment1")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Feedforward neural network



In [ ]:
import wandb
from keras.datasets import fashion_mnist
import numpy as np

In [81]:
"""
  ACTIVATION FUNCTIONS
"""
def identity(x):
    return x

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Numerical Stability
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [99]:
"""
  LOSS FUNCTIONS
"""
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

import numpy as np

def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred), axis=-1)

In [ ]:
class FeedForwardNeuralNetwork:
  # class variables
  optimizersMap = {"a":lambda x: x+1}
  lossFunctionsMap = {"a":lambda x: x+1}
  activationFunctionsMap = {"a":lambda x: x+1}
  derivatesFuncMap = {"a": lambda x: x+1}

  def __init__(self,
               input_size=784, output_size=10,
               n_hiddenLayers=3, n_neuronsPerLayer=32,
               activationFun="sigmoid",
               weight_init="random",
               batch_size=64,
               lossFunc="cross_entropy",
               optimizer="adam",
               learning_rate=0.001,
               momentum=0.5,
               beta=0.9, beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.01,
               epochs=10):

    # Inialtization parameters
    self.input_size = input_size  # no of features
    self.output_size = output_size
    self.n_hiddenLayers = n_hiddenLayers
    self.n_neuronsPerLayer = n_neuronsPerLayer
    self.weight_init = weight_init
    self.epochs = epochs

    self.activationFun = FeedForwardNeuralNetwork.activationFunctionsMap[activationFun]
    self.lossFunc = FeedForwardNeuralNetwork.lossFunctionsMap[lossFunc]
    self.optimizer = FeedForwardNeuralNetwork.optimizersMap[optimizer]

    # paramters required for optimizers
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.beta = beta
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.weight_decay = weight_decay

    # weights and biases matrices
    self.weights = []
    self.biases = []
    self.initializeWeightsAndBiases()

    # pre-activation(a) and post-activation(h) values
    self.a = []
    self.h = []

  '''
    Weights,Biases initialization based on weight_init parameter

    weights[0]: input layer to first hidden layer  : input_size x n_neuronsPerLayer
    weights[1]: first hidden layer to second hidden layer : n_neuronsPerLayer x n_neuronsPerLayer
    ...
    weights[n_hiddenLayers]: last hidden layer to output layer : n_neuronsPerLayer x output_size

    biases[i] : bias for ith layer : 1 x n_neuronsPerLayer   (i:0 to n_hiddenLayers-1)
    biases[n_hiddenLayers]: 1 x output_size
  '''
  def initializeWeightsAndBiases(self):
    # biases for both types
    for _ in range(self.n_hiddenLayers):
      self.biases.append(np.zeros(self.n_neuronsPerLayer))
    self.biases.append(np.zeros(self.output_size))   # biases[n_hiddenLayers]

    if(self.weight_init == "random"):   # Random Normal
      # weights[0]
      self.weights.append(np.random.randn(self.input_size, self.n_neuronsPerLayer))

      # weights[1] -> weights[n_hiddenLayers-1]
      for _ in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.n_neuronsPerLayer))

      # weights[n_hiddenLayers]
      self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.output_size))
    elif(self.weight_init == "Xavier"):   # Xavier Normal: mean = 0, variance = 2/(n_input + n_output)
      # weights[0]
      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.input_size + self.n_neuronsPerLayer)), size=(self.input_size, self.n_neuronsPerLayer)))

      for _ in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.n_neuronsPerLayer)), size=(self.n_neuronsPerLayer, self.n_neuronsPerLayer)))

      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.output_size)), size=(self.n_neuronsPerLayer, self.output_size)))

  '''
    Forward propagation through the neural network. (for batch)
    Instead of doing one input at a time, this function handles it for a batch using respective sized matrices

    x_batch: B x n where B - batch size, n- no of features = input_size
    x_batch is assumbed to be numpy array when given as input
  '''
  def forwardPropagation(self, x_batch):
    a_pre_activation = []
    h_post_activation = []

    # considering a0,h0 as X values as a1: first layer  (it is calculated from x values)
    a_pre_activation.append(x_batch)
    h_post_activation.append(x_batch)

    # Except last layer since activation function could be different
    for i in range(self.n_hiddenLayers):
      # ai: B x n_neuronsPerLayer, biases[i]: 1 x n_neuronsPerLayer (it will be broadcasted while adding)
      ai = np.matmul(h_post_activation[-1], self.weights[i]) + self.biases[i]
      hi = self.activationFun(ai)

      a_pre_activation.append(ai)
      h_post_activation.append(hi)

    # aL: last layer (activation function is softmax)
    aL = np.matmul(h_post_activation[-1], self.weights[self.n_hiddenLayers]) + self.biases[self.n_hiddenLayers]
    hL = softmax(aL)   # y_batch

    a_pre_activation.append(aL)
    h_post_activation.append(hL)

    return a_pre_activation, h_post_activation

  '''
    Backward propagation through the neural network. (for batch)
  '''
  def backwardPropagation(self, a_pre_activation, h_post_activation, y_batch, y_pred_batch):
    grad_w = []
    grad_b = []
    grad_a = []
    grad_h = []

    # Output gradient (wrt aL)
    grad_hL = self.derivatesFuncMap(self.lossFunc)(y_batch, y_pred_batch)
    pass

In [6]:
import numpy as np

# # Create a column vector (shape: (3,1))
# v1 = np.array([[1,2], [2,3], [3,4]])  # Shape (3,1)
# v2 = np.array([[4], [5], [6]])  # Shape (3,1)

# # Append to a list
# vector_list = []
# vector_list.append(v1)
# vector_list.append(v2)

# print(vector_list)

# # flatten each element of x_train, after flattening store it as np array
# x_ftrain = np.array([x.flatten() for x in x_train])
# x1 = [x.flatten() for x in x_train]
# # print(x_ftrain[0].shape)
# # print(x_ftrain.shape)
# # print(x_ftrain[:20].shape)
# # print(y_train[:20].shape)
# # print(x_test.shape)
# # print(y_test.shape)
# m = np.array(np.zeros((784,3)))
# y = np.matmul(x_ftrain[:20],m)
# print(y.shape)


y_batch = np.array([
    [0, 1, 0.5],  # Example 1
    [1, 0, 0],  # Example 2
    [0, 0, 1]   # Example 3
])

y_hat_batch = np.array([
    [0.2, 0.5, 0.3],
    [0.8, 0.1, 0.1],
    [0.3, 0.3, 0.4]
])

grad_batch = - (y_batch / y_hat_batch)  # Element-wise division
print(grad_batch)


[[-0.         -2.         -1.66666667]
 [-1.25       -0.         -0.        ]
 [-0.         -0.         -2.5       ]]


In [93]:
import numpy as np

def identity(x):
    return x

def identity_derivative(x):
    return np.ones_like(x)

x_single = 2.0
x_vector = np.array([1.0, 2.0, -3.0])
x_batch = np.array([[1.0, 1.0], [3.0, 4.0]])

print(identity(x_batch))  # Works
print(identity_derivative(x_batch))  # Works


[[1. 1.]
 [3. 4.]]
[[1. 1.]
 [1. 1.]]


In [66]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

print(sigmoid(x_vector))  # Works
print(sigmoid_derivative(x_vector))  # Works

[0.73105858 1.         0.04742587]
[0.19661193 0.         0.04517666]


In [67]:
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

print(tanh(x_vector))  # Works
print(tanh_derivative(x_single))  # Works

[ 0.76159416  1.         -0.99505475]
0.07065082485316443


In [68]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

print(relu(x_vector))  # Works
print(relu_derivative(x_vector))  # Works

[1.e+00 2.e+03 0.e+00]
[1 1 0]


In [74]:
# # def softmax(x):
# #     exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Stability trick
# #     return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# # Softmax needs a vector input
# x_vector = np.array([1.0, 20000.0, 3.0])
# x_batch = np.array([[1.0, 2.0, 3000.0], [1.5, 2.5, 3.5]])

# print(softmax(x_vector))  # Works
# print(softmax(x_batch))  # Works (batch-wise softmax)

In [39]:
print(y_train)

[9 0 0 ... 3 0 5]


In [75]:
import numpy as np

num_classes = 10
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.eye(num_classes)[y_test]

print(y_train_one_hot[-1])  # Output: (num_samples, 10)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [95]:
print(x_vector)
x_vector=[1,2,3]

[ 1.  2. -3.]
